## 模型（Model）与层（Layer）

将模型编写为类，然后在模型调用的地方使用`y_pred = model(x)`的形式进行调用。**模型类**的形式非常简单，主要包含`_init__()`（构造函数，初始化）和`call(input)`（模型调用）两个方法，但也可以根据需要增加自定义的方法。
```
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # 此处添加初始化代码（包含call方法中会用到的层）
    
    def call(self, inputs):
        # 此处添加模型调用的代码（处理输入并返回输出）
        return output
```

这里我们的模型继承了tf.kears.Model。继承tf.keras.Model的一个好处在于我们可以使用父类的若干方法和属性，例如在实例化类后可以通过model.variables这一属性直接获得模型中的所有变量，免得我们一个个显式指定变量的麻烦。

同时，我们引入“层”（Layer）的概念，层可以视为比模型粒度更细的组件单位，将计算流程进行了封装。我们可以使用层来快速搭建模型。

`y_pred = tf.matmul(X, w) + b`可以通过模型类的方式编写如下：

In [15]:
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

In [9]:
X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])

In [10]:
class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(units=1, kernel_initializer=tf.zeros_initializer(),
                                           bias_initializer=tf.zeros_initializer())
    
    def call(self, input):
        output = self.dense(input)
        return output

In [14]:
# 实例化Linear类
model = Linear()
# 声明一个梯度下降优化器
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
for i in range(100):
    # 使用tf.GradientTape()记录损失函数的梯度信息
    with tf.GradientTape() as tape:
        # 调用模型
        y_pred = model(X)
        loss = tf.reduce_mean(tf.square(y_pred - y))
    # 自动计算自变量的梯度
    grads = tape.gradient(loss, model.variables)
    # 自动根据梯度更新参数
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
print(model.variables)

[<tf.Variable 'linear_4/dense_4/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.40784496],
       [1.191065  ],
       [1.9742855 ]], dtype=float32)>, <tf.Variable 'linear_4/dense_4/bias:0' shape=(1,) dtype=float32, numpy=array([0.78322077], dtype=float32)>]


这里，我们没有显式地声明`w`和`b`两个变量并写出`y_pred = tf.matmul(X, w) + b`这一线性变换，而是在初始化部分实例化了一个全连接层（`tf.keras.layers.Dense`），并在call方法中对这个层进行调用。全连接层封装了`output = activation(tf.matmul(input, kernel) + bias)`这一线性变换+激活函数的计算操作，以及`kernel`和`bias`两个变量。当不指定激活函数时（即`activaion(x) = x`），这个全连接层就等价于我们上述的线性变换。

## 基础示例：多层感知机（MLP）

我们从编写一个最简单的多层感知机（Multilayer Perceptron，MLP）开始，介绍TensorFlow的模型编写方式。这里我们使用多层感知机完成MNIST手写体数字图片数据集的分类任务。

先进行预备工作，实现一个简单的`DataLoader`类来读取MNIST数据集数据。

In [38]:
class DataLoader():
    def __init__(self):
        mnist = tf.contrib.learn.datasets.load_dataset('mnist')
        self.train_data = mnist.train.images                               # np.array [55000, 784] 
        self.train_labels = np.asarray(mnist.train.labels, dtype=np.int32)  # np.array [55000] of int32
        self.eval_data = mnist.test.images                                  # np.array [10000, 784]
        self.eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)    # np.array [10000] of int32
        
    def get_batch(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_labels[index]

多层感知机的模型类实现与上面的线性模型类似，所不同的地方在于层数增加了（顾名思义，“多层”感知机），以及引入了非线性激活函数（这里使用了ReLU函数，即下方的`activation=tf.nn.relu`）。该模型输入一个向量（比如这里是拉直的1×784手写体数字图片），输出10维的信号，分别代表这张图片属于0到9的概率。这里我们加入了一个predict方法，对图片对应的数字进行预测。在预测的时候，选择概率最大的数字进行预测输出。

**线性整流函数**（Rectified Linear Unit，**ReLU**）,又称修正线性单元, 是一种人工神经网络中常用的激活函数（activation function），通常指代以斜坡函数及其变种为代表的非线性函数。  
比较常用的线性整流函数有斜坡函数$f(x)=max(0,x)$，以及带泄露整流函数 (Leaky ReLU)。  
$f(x)=max(0,x)$又可写成$f(x)=\begin{cases}x& \text{x}{\geq}{0;}\\0& \text{x}{\lt}{0}\end{cases}$

In [40]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self, inputs):
        x = self.dense1(inputs)
        y = self.dense2(x)
        return x
    
    def predict(self, inputs):
        logits = self(inputs)
        return tf.argmax(logits, axis=-1)

定义一些模型超参数：

In [41]:
num_batches = 1000
batch_size = 50
learning_rate = 0.001

实例化模型，数据读取类和优化器：

In [45]:
model = MLP()
data_loader = DataLoader()
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz


然后迭代进行以下步骤：
* 从DataLoader中随机取一批训练数据；
* 将这批数据送入模型，计算出模型的预测值；
* 将模型预测值与真实值进行比较，计算损失函数（loss）；
* 计算损失函数关于模型变量的导数；
* 使用优化器更新模型参数以最小化损失函数。

In [46]:
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_logit_pred = model(tf.convert_to_tensor(X))
        loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_logit_pred)
        if batch_index % 9 == 0:
            print('batch %d: loss %f' % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 4.704428
batch 9: loss 4.003401
batch 18: loss 3.584399
batch 27: loss 2.823752
batch 36: loss 2.561762
batch 45: loss 2.677138
batch 54: loss 1.891707
batch 63: loss 2.126160
batch 72: loss 1.846991
batch 81: loss 1.849578
batch 90: loss 1.490038
batch 99: loss 1.626411
batch 108: loss 1.321092
batch 117: loss 1.195409
batch 126: loss 2.006490
batch 135: loss 2.017419
batch 144: loss 1.338521
batch 153: loss 1.215853
batch 162: loss 1.122013
batch 171: loss 1.595369
batch 180: loss 1.038400
batch 189: loss 1.428598
batch 198: loss 1.561793
batch 207: loss 1.073895
batch 216: loss 1.192201
batch 225: loss 0.984081
batch 234: loss 1.609973
batch 243: loss 1.095677
batch 252: loss 1.887974
batch 261: loss 0.893708
batch 270: loss 1.444066
batch 279: loss 1.179849
batch 288: loss 1.277989
batch 297: loss 0.976289
batch 306: loss 1.266387
batch 315: loss 1.156502
batch 324: loss 1.504180
batch 333: loss 1.128154
batch 342: loss 0.899152
batch 351: loss 1.213965
batch 360: los

接下来，我们使用验证集测试模型性能。具体而言，比较验证集上模型预测的结果与真是结果，输出预测正确的样本数占总样本数的比例：

In [47]:
num_eval_samples = np.shape(data_loader.eval_labels)[0]
y_pred = model.predict(data_loader.eval_data).numpy()
print('test accuracy: %f' % (sum(y_pred == data_loader.eval_labels) / num_eval_samples))

test accuracy: 0.823000


## 卷积神经网络（CNN）

卷积神经网络（Convolutional Neural Network，CNN）是一种结构类似于人类或动物的视觉系统的人工神经网络，包含一个或多个卷积层（Convolutional Layer）、池化层（Pooling Layer）和全连接层（Dense Layer）。

In [59]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,  # 卷积核数目
            kernel_size=[5, 5],  # 感受野大小
            padding='same',  # padding策略
            activation=tf.nn.relu  # 激活函数
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self, inputs):
        inputs = tf.reshape(inputs, [-1, 28, 28, 1])
        x = self.conv1(inputs)  # [batch_size, 28, 28,32]
        x = self.pool1(x)  # [batch_size, 14, 14, 32]
        x = self.conv2(x)  # [batch_size, 14, 14, 64]
        x = self.pool2(x)  # [batch_size, 7, 7, 64]
        x = self.flatten(x)  # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)  # [batch_size, 1024]
        x = self.dense2(x)  # [batch_size, 10]
        return x
    
    def predict(self, inputs):
        logits = self(inputs)
        return tf.argmax(logits, axis=-1)

将前面的`model = MLP()`更换成`model = CNN()`：

In [64]:
model = CNN()
data_loader = DataLoader()
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz


In [65]:
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_logit_pred = model(tf.convert_to_tensor(X))
        loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_logit_pred)
        if batch_index % 111 == 0:
            print('batch %d: loss %f' % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.296304
batch 111: loss 0.080120
batch 222: loss 0.124463
batch 333: loss 0.020256
batch 444: loss 0.063452
batch 555: loss 0.232355
batch 666: loss 0.018755
batch 777: loss 0.119675
batch 888: loss 0.019599
batch 999: loss 0.136880


In [66]:
y_pred = model.predict(data_loader.eval_data).numpy()
print('test accuracy: %f' % (sum(y_pred == data_loader.eval_labels) / num_eval_samples))

test accuracy: 0.982900


## 循环神经网络（RNN）

循环神经网络（Recurrent Neural，RNN）是一种适宜于处理序列数据的神经网络，被广泛用于语言模型、文本生成、机器翻译等。

这里，我们使用RNN来进行尼采风格文本的自动生成。  
这个任务的本质其实预测一段英文文本的接续字母的概率分布。比如，我们有以下句子：
```
I am a studen```

这个句子（序列）一共有13个字符（包含空格）。当我们阅读到这个由13个字符组成的序列后，根据我们的经验，我们可以预测出下一个字符很大概率是“t”。

In [ ]:
class DataLoader():
    def __init__(self):
        path